In [ ]:
specs = {'adaptingDataframe': {'changeType': {}, 'renameMap': {}, 'setValue': None, 'colName': 'id_indicateur',
                               'value': '??', 'toDrop': []}, 'catalogUri': 'None',
         'confDb': {'chunksize': 1000, 'schema': 'surfor', 'strategy': 'append',
                    'tableName': 'faits_GFC_treecover2000'},
         'confDims': {'isin_id_mesure': [3, 100], 'isin_id_spatial': ['None']},
         'confRaster': {'api': 'GEE', 'masque': None, 'outputValue': 'continue', 'defaultValue': 0.0,
                        'overlayHow': 'identity', 'uri_image': 'UMD/hansen/global_forest_change_2021_v1_9',
                        'windows_height': 2048, 'windows_width': 2048, 'bandName': 'treecover2000'}, 'dataName': 'gfc_treecover2000',
         'epsg': 'EPSG:4326', 'overlayHow': 'identity', 'sourceType': 'Raster'}

In [ ]:
%load_ext autoreload
%autoreload 2
import ee
import geemap
import sys
sys.path.append("..")
from generate_indicator import gee
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show

# instanciation de ee
service_account = 'ee-oeil@surfor.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '../generate_indicator/gee_credentials.json')
ee.Initialize(credentials)

# création de l'input : geodataframe avec N polygone et N points
point1 = ee.Geometry.Point((166.605758, -22.127171)) # en pleine foret
point2 = ee.Geometry.Point((166.450659, -22.284149)) # en pleine ville
features = [
    ee.Feature(point1, {'name': 'point1', 'description': 'en pleine foret'}),
    ee.Feature(point2, {'name': 'point2', 'description': 'en pleine ville'}),
]
gdf = geemap.ee_to_geopandas(ee.FeatureCollection(features)).set_crs("EPSG:4326")

# traitement
output_gdf = gee.extract_data(specs, gdf)

In [ ]:
for feature in output_gdf.iterfeatures():
    pixel_value = feature.get('properties').get('treecover2000')
    print("Le point '{name}' ({description}) a une valeur de '{valeur}'".format(name=feature.get('properties').get('name'), valeur=pixel_value, description=feature.get('properties').get('description')))